In [6]:
import pandas as pd
from data import CassavaDataset
from pathlib import Path
from augmentations import get_augmentations,get_tta
from torch.utils.data import DataLoader
from lightning import CassavaModel
import torch
from models import Resnext

In [7]:
test_df = pd.read_csv('../data/sample_submission.csv')
path = Path('../data/')
batch_size,num_workers = 32,8
ssl_models = [
    "resnet18_ssl",
    "resnet50_ssl",
    "resnext50_32x4d_ssl",
    "resnext101_32x4d_ssl",
    "resnext101_32x8d_ssl",
    "resnext101_32x16d_ssl",
]

In [8]:
tta_tfms = get_tta(image_size=512)
test_ds = CassavaDataset(path=path/'test_images',df=test_df,transform=tta_tfms)
test_df = pd.read_pickle('../data/valid_df.pkl')
test_ds = CassavaDataset(path=path/'train_images',df=test_df,transform=tta_tfms)


In [9]:
test_dl = DataLoader( dataset=test_ds,
            batch_size=batch_size,
            num_workers=num_workers,
            shuffle=False,
            pin_memory=True,)

In [10]:
device = torch.device('cuda')
model = Resnext(model_name=ssl_models[2],num_classes=5,kaggle=True)
model = model.to(device)


HI


In [28]:
ls -al Cassava/3k3rrswa/checkpoints/epoch=9-step=449.ckpt

total 269900
drwxr-xr-x 2 root root        35 Dec 14 16:11  ./
drwxr-xr-x 3 root root        25 Dec 14 15:41  ../
-rw-r--r-- 1 root root 276374145 Dec 14 16:11 'epoch=9-step=449.ckpt'


In [29]:
chk_path = 'Cassava/3k3rrswa/checkpoints/epoch=9-step=449.ckpt'
chk = torch.load(chk_path)
model_weights = {k.replace('model.',''):v for k,v in chk['state_dict'].items()}
torch.save(model_weights,'model_weights.pth')

model.load_state_dict(model_weights)

<All keys matched successfully>

In [14]:
def get_preds():
    preds = []
    with torch.no_grad():
        for xb,_ in test_dl:
            xb = xb.to(device)
            pred = model(xb)
            preds.append(pred.to('cpu'))
    return torch.cat(preds)




In [15]:
preds = get_preds()

In [16]:
preds.shape

torch.Size([4280, 5])

In [18]:
preds = torch.zeros(len(test_ds),5)
for o in range(2):
    preds += get_preds()

In [20]:
preds /= 2

In [24]:
preds.argmax(1).tolist()

[3,
 3,
 3,
 2,
 3,
 4,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 4,
 2,
 3,
 1,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 1,
 3,
 3,
 0,
 3,
 4,
 3,
 3,
 3,
 4,
 1,
 3,
 0,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 0,
 2,
 1,
 2,
 3,
 3,
 1,
 3,
 3,
 3,
 1,
 3,
 1,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 2,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 2,
 1,
 3,
 3,
 4,
 3,
 4,
 0,
 3,
 2,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 4,
 1,
 4,
 3,
 4,
 0,
 0,
 1,
 3,
 3,
 2,
 3,
 1,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 2,
 2,
 4,
 3,
 3,
 0,
 2,
 2,
 4,
 3,
 1,
 4,
 2,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 2,
 3,
 1,
 3,
 3,
 4,
 2,
 3,
 3,
 2,
 4,
 3,
 1,
 0,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 4,
 3,
 3,
 3,
 4,
 3,
 1,
 3,
 2,
 3,
 0,
 3,
 0,
 2,
 3,
 3,
 3,
 2,
 4,
 4,
 3,
 3,
 4,
 3,
 1,
 3,
 3,
 0,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 2,
 3,
 1,
 3,
 1,
 4,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 1,
 4,
 2,
 4,
 3,
 3,
 3,
 3,
 4,
 4,
 0,
 3,


In [16]:
test_df.head()

,image_id,label
10544,288080098.jpg,3
11634,3080364100.jpg,3
578,110051175.jpg,3
9118,2612067247.jpg,1
15857,3852927202.jpg,3
